# Clone a repo

The calculation of FLOPS and MAACs is based on this repository: https://github.com/ckyrkou/Keras_FLOP_Estimator

In [1]:
!git clone https://github.com/ckyrkou/Keras_FLOP_Estimator.git

Cloning into 'Keras_FLOP_Estimator'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 176 (delta 3), reused 0 (delta 0), pack-reused 165
Receiving objects: 100% (176/176), 140.48 KiB | 1.76 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [2]:
%cp /content/net_flops.py /content/Keras_FLOP_Estimator/python_code

I made a change to the file located at:

*Keras_FLOP_Estimator/python_code/net_flops.py*

This change reports MAACs for each layer.

The modified file is located in this repository. Please copy and replace it at:

*Keras_FLOP_Estimator/python_code/net_flops.py*

In [1]:
%cd /content/Keras_FLOP_Estimator/python_code

/content/Keras_FLOP_Estimator/python_code


# Imports

In [2]:
from net_flops import net_flops
import tensorflow as tf

# Configs

In [3]:
class configs:
  img_size = (224, 224)
  num_classes = 10
  num_most_complex_layers = 10 #Specify the number of most computationally expensive layers to get

# Model

In [4]:
def get_model(configs):
  """Gets a Keras model.

  Args:
    configs: Configurations class

  Returns:
    model: Keras model
  """

  IMG_SHAPE = configs.img_size + (3,)
  base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
  base_model.trainable = False

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

  prediction_layer = tf.keras.layers.Dense(configs.num_classes, activation='softmax')

  x = base_model.output
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  outputs = prediction_layer(x)
  model = tf.keras.Model(base_model.input, outputs)
  print(model.summary())
  return model

In [5]:
model = get_model(configs)

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                               

# Calculate FLOPS and MAACs

In [6]:
def get_most_complex_layers(layers_flops, num_layers):
  """Gets the most computationally expensive layers.

  Args:
    layers_flops: A dictionary where keys are the layers and values are FLOPS for each layer.
    num_layers: The number of most computationally expensive layers to get.

  Returns:
    sorted_layers_flops: A dictionary with the most computationally expensive layers
    where keys are the layers and values are FLOPS for each layer.
  """

  sorted_layers_flops = sorted(layers_flops.items(), key=lambda x: x[1], reverse=True)
  return sorted_layers_flops[:num_layers]

In [7]:
layers_flops = net_flops(model,table=True)

               Layer Name |      Input Shape |     Output Shape |      Kernel Size |          Filters | Strides |  FLOPS |  MAACS
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                  input_1 |    [224, 224, 3] |    [224, 224, 3] |           [0, 0] |           [0, 0] | [1, 1] | 0.0000 | 0.0000
                    Conv1 |    [224, 224, 3] |   [112, 112, 32] |           (3, 3) |               32 | (2, 2) | 21676032.0000 | 10838016.0000
                 bn_Conv1 |   [112, 112, 32] |   [112, 112, 32] |           [0, 0] |           [0, 0] | [1, 1] | 0.0000 | 0.0000
               Conv1_relu |   [112, 112, 32] |   [112, 112, 32] |           [0, 0] |           [0, 0] | [1, 1] | 0.0000 | 0.0000
  expanded_conv_depthwise |   [112, 112, 32] |   [112, 112, 32] |           (3, 3) |               32 | (1, 1) | 7225344.0000 | 3612672.0000
expanded_conv_depthwise_BN |

In [8]:
most_complex_layers = get_most_complex_layers(layers_flops, configs.num_most_complex_layers)

In [9]:
print("Most {} computationally expensive layers:\n".format(configs.num_most_complex_layers))
most_complex_layers

Most 10 computationally expensive layers:



[('Conv_1', 40140800.0),
 ('block_1_expand', 38535168.0),
 ('block_16_project', 30105600.0),
 ('Conv1', 21676032.0),
 ('block_2_expand', 21676032.0),
 ('block_2_project', 21676032.0),
 ('block_3_expand', 21676032.0),
 ('block_11_expand', 21676032.0),
 ('block_11_project', 21676032.0),
 ('block_12_expand', 21676032.0)]